# Traffic sign detection and classification

In [4]:
from xml.dom import minidom
from os import walk
import pandas as pd
import cv2 as cv
import numpy as np
from sklearn import metrics
import os

In [5]:
filenames = next(walk("res/annotations"), (None, None, []))[2]  # [] if no file

signs = []

for annotation in filenames:
    # parse an xml file by name
    file = minidom.parse("res/annotations/" + annotation)
    #use getElementsByTagName() to get tag
    if len(file.getElementsByTagName('name')) == 1:
        path = "res/images/" + file.getElementsByTagName('filename')[0].firstChild.data
        filename = file.getElementsByTagName('filename')[0].firstChild.data
        name = file.getElementsByTagName('name')[0].firstChild.data
        # truncated = file.getElementsByTagName('truncated')[0].firstChild.data
        # occluded = file.getElementsByTagName('occluded')[0].firstChild.data
        # difficult = file.getElementsByTagName('difficult')[0].firstChild.data

        if name == "trafficlight":
            continue

        signs.append([filename, name, path])

print(f'Importing {len(signs)} images')

df = pd.DataFrame(signs, columns=['filename', 'name', 'path'])


Importing 588 images


In [6]:
def condition_classes(s):
    if s['name'] == 'speedlimit':
        return 0
    elif s["name"] == 'crosswalk':
        return 1
    elif s["name"] == "stop":
        return 2


df["class"] = df.apply(condition_classes, axis=1)

## Step 1 - Histogram equalization

TODO - Need to improve the histogram equalization

In [7]:
def apply_histogram_equalization(row):
    img = cv.imread(row.path)
    lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)

    clahe = cv.createCLAHE(clipLimit=10.0,tileGridSize=(8,8))

    lab[...,0] = clahe.apply(lab[...,0])

    out = cv.cvtColor(lab, cv.COLOR_LAB2BGR)
    
    #para ignorar este passo 
    #out = img

    cv.imwrite("output2/1histogram/" + row.filename, out)

In [8]:
df.apply(apply_histogram_equalization, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
583    None
584    None
585    None
586    None
587    None
Length: 588, dtype: object

## Step 2 - Segmentation by Color

In [9]:
def apply_segmentation(row):
    img = cv.imread(row.path)
    # TODO - work on histogram equalization
    img_hist = cv.imread("output2/1histogram/" + row.filename)
    img_hsv = cv.cvtColor(img_hist, cv.COLOR_BGR2HSV)

    lower_red_m1 = (0, 70, 30)
    upper_red_m1 = (10, 255, 255)

    lower_red_m2 = (170, 70, 30)
    upper_red_m2 = (180, 255, 255)

    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 200)
    
    mask1 = cv.inRange(img_hsv, lower_red_m1, upper_red_m1)
    mask2 = cv.inRange(img_hsv, lower_red_m2, upper_red_m2)

    blue_mask = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)
    
    red_mask = mask1 + mask2

    # out = cv.bitwise_and(img_hist, img_hist, mask=mask)
    if cv.countNonZero(red_mask) !=0 :
        cv.imwrite("output2/2segmentation/red/" + row.filename, red_mask)
    if cv.countNonZero(blue_mask) != 0:
        cv.imwrite("output2/2segmentation/blue/" + row.filename, blue_mask)
        


In [10]:
df.apply(apply_segmentation, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
583    None
584    None
585    None
586    None
587    None
Length: 588, dtype: object

## Step 3 - Post-Processing

In [11]:
def compare_masks(blue, red): 
    return blue if red == np.zeros(red.shape) else (red if blue == np.zeros(blue.shape) else blue, red)

In [12]:
def apply_post_processing_blue(row):
    img = cv.imread(row.path)
    segm_img = cv.imread("output2/2segmentation/blue/" + row.filename, cv.IMREAD_GRAYSCALE)
    if cv.countNonZero(segm_img) == 0 :
        return

    linesP = cv.HoughLinesP(segm_img, 1, np.pi / 180, 50, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(segm_img, (l[0], l[1]), (l[2], l[3]),
                    (255, 0, 255), 3, cv.LINE_AA)

    # blue 
    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    rows, cols = out.shape

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    mask = np.full(img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    
    # morphological operations
    out = cv.erode(mask, kernel, iterations=1)
    out = cv.dilate(mask, kernel, iterations=5)

    cv.imwrite("output2/3post_processing/blue/" + row.filename, out)


In [13]:
df.apply(apply_post_processing_blue, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
583    None
584    None
585    None
586    None
587    None
Length: 588, dtype: object

In [14]:
def apply_post_processing_red(row):
    img = cv.imread(row.path)
    red_segm_img = cv.imread(
        "output2/2segmentation/red/" + row.filename, cv.IMREAD_GRAYSCALE)

    # apply median filter to remove noise
    out = cv.medianBlur(red_segm_img, 5)
    rows, cols = out.shape

     # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=3)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    mask = np.full(img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    # morphological operations

    cv.imwrite("output2/3post_processing/red/" + row.filename, out)


In [15]:
df.apply(apply_post_processing_red, axis=1);

## Step 4 - Shape Recognition

In [16]:
def get_contours(img, processed):
    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_SIMPLE)
    contours, _ = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(
            contour, 0.01*cv.arcLength(contour, True), True)
        sh = ""
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "R"
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "O"
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "C"

        processed = cv.drawContours(processed, [contour], 0, (128, 0, 255), 2)
        
        processed = cv.putText(img=processed, text=sh, org=approx[0][0],
                            fontFace=cv.FONT_ITALIC,
                            fontScale=1.0,
                            color=(128, 0, 255),
                            thickness=1)
        

    return processed, shapes

In [17]:
def shape_recognition(row):
    img = cv.imread(row.path)
    processed_blue = cv.imread(
        "output2/3post_processing/blue/" + row.filename, 0)
    processed_red = cv.imread(
        "output2/3post_processing/red/" + row.filename, 0)

    if processed_blue is not None:
        contours_blue, shapes_blue = get_contours(img, processed_blue)

    if processed_red is not None : 
        contours_red, shapes_red = get_contours(img, processed_red)

    shapes = shapes_blue + shapes_red

    #contours = cv.hconcat((contours_blue, contours_red))
    cv.imwrite("output2/4contours/red/" + row.filename, contours_red)
    cv.imwrite("output2/4contours/blue/" + row.filename, contours_blue)

    return shapes


In [18]:
df["shapes"] = df.apply(shape_recognition, axis=1);

In [19]:
def classify(row):
    shapes_info = row.shapes
    if len(shapes_info) == 0:
        return -1

    shapes = [x[0] for x in shapes_info]
    shape = shapes[0]

    if shape == 'circle':
        return 0
    elif shape == 'rectangle':
        return 1
    elif shape == 'octagon':
        return 2

    return -1

df["classification"] = df.apply(classify, axis=1);

In [20]:
def concatenate_and_write(row):
    img = cv.imread(row.path)
    #hist = cv.imread("output2/1histogram/red/" + row.filename)
    hist = img
    b_segm = cv.imread("output2/2segmentation/blue/" + row.filename)
    b_post = cv.imread("output2/3post_processing/blue/" + row.filename)
    r_segm = cv.imread("output2/2segmentation/red/" + row.filename)
    r_post = cv.imread("output2/3post_processing/red/" + row.filename)
    b_cnt = cv.imread("output2/4contours/blue/" + row.filename)
    r_cnt = cv.imread("output2/4contours/red/" + row.filename)
    vis = np.concatenate((img, hist, b_segm, r_segm, b_post, r_post, b_cnt, r_cnt), axis=1)

    result = "right" if row['class'] == row['classification'] else "wrong"

    c = df['class'][row.name]
    p = os.path.join("output2", "5concatenated",
                     f'classe{c}', result + "_" + row.filename)
    cv.imwrite(p, vis)


In [21]:
df.apply(concatenate_and_write, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
583    None
584    None
585    None
586    None
587    None
Length: 588, dtype: object

## Results

In [22]:
df_found = df[df["classification"] != -1]

accuracy_total = metrics.accuracy_score(df["class"], df["classification"])
accuracy_found = metrics.accuracy_score(df_found["class"], df_found["classification"])

print("Classes:\n", df["class"].value_counts())
print("Detected classes:\n", df["classification"].value_counts())
print("Accuracy Total: {:.02f}%".format(accuracy_total*100))
print("Accuracy Signs Found: {:.02f}%".format(accuracy_found*100))

Classes:
 0    481
2     65
1     42
Name: class, dtype: int64
Detected classes:
  0    498
 2     51
 1     28
-1     11
Name: classification, dtype: int64
Accuracy Total: 79.42%
Accuracy Signs Found: 80.94%


In [23]:
failed_0 = len(df[(df["class"] == 0) & (df["classification"] != 0)])    # speedlimit
failed_1 = len(df[(df["class"] == 1) & (df["classification"] != 1)])    # crosswalk
failed_2 = len(df[(df["class"] == 2) & (df["classification"] != 2)])    # stop

failed_1_0 = len(df[(df["class"] == 1) & (df["classification"] == 0)])
failed_1_2 = len(df[(df["class"] == 1) & (df["classification"] == 2)])

right_0 = len(df[(df["class"] == 0) & (df["classification"] == 0)])
right_1 = len(df[(df["class"] == 1) & (df["classification"] == 1)])
right_2 = len(df[(df["class"] == 2) & (df["classification"] == 2)])

not_classified_0 = len(df[(df["class"] == 0) & (df["classification"] == -1)])
not_classified_1 = len(df[(df["class"] == 1) & (df["classification"] == -1)])
not_classified_2 = len(df[(df["class"] == 2) & (df["classification"] == -1)])

print("Class 0 - SpeedLimit R:{}|W:{}|NC:{}".format(right_0, failed_0, not_classified_0))
print("Class 1 - Crosswalk: R:{}|W:{}|NC:{}|0-Circle:{}|2-Octagon:{}".format(right_1, failed_1, not_classified_1, failed_1_0,failed_1_2))
print("Class 2 - Stop: R:{}|W:{}|NC:{}".format(right_2, failed_2, not_classified_2))


Class 0 - SpeedLimit R:434|W:47|NC:9
Class 1 - Crosswalk: R:11|W:31|NC:2|0-Circle:26|2-Octagon:3
Class 2 - Stop: R:22|W:43|NC:0


In [24]:
import cv2 as cv
import numpy as np
import skimage as sk

def apply_post_processing_red(img):
    segm_img = cv.imread("output2\\2segmentation\\red\\road56.png", 0)
    cv.imshow("out", segm_img)
    cv.waitKey(0)
    cv.destroyWindow("out")

    # blue
    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    
    # Taking a matrix of size 5 as the kernel
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))
    kernel = sk.morphology.disk(3)
    print(kernel)
    

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=6)

    cv.imshow("blur", out)
    cv.waitKey(0)
    cv.destroyWindow("blur")

    detected_circles = cv.HoughCircles(out,
                                        cv.HOUGH_GRADIENT, 1, 20, param1=50,
                                        param2=20, minRadius=1, maxRadius=40)

    # Draw circles that are detected.
    if detected_circles is not None:
        print("detected")
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))

        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]

            # Draw the circumference of the circle.
            cv.circle(out, (a, b), r, (128, 255, 0), 2)

    cv.imshow("last", out)
    cv.waitKey(0)
    cv.destroyWindow("last")

    #cv.imwrite("output2/3post_processing/blue/road140.png", out)

apply_post_processing_red("ss")


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'out' in function 'cvDestroyWindow'


In [ ]:
def shape_recognition():
    processed = cv.imread("output2\\2segmentation\\red\\road56.png", 0)
    cv.imshow("out", processed)
    cv.waitKey(0)
    cv.destroyWindow("out")

    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_SIMPLE)
    contours, h = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]

    #contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    print(h)

    for i, contour in enumerate(contours):
        print(h[0][i][3])
        if float(cv.contourArea(contour) / (processed.shape[0]*processed.shape[1])) >= 0.95:
            continue
        processed_1 = np.ones(processed.shape[:2], dtype="uint8")
        approx = cv.approxPolyDP(
            contour, 0.01*cv.arcLength(contour, True), True)
        cv.drawContours(processed, [contour], 0, (128, 0, 255), 2)
        cv.drawContours(processed_1, [contour], 0, (128, 0, 255), 2)
        print(len(approx))

        if len(approx) > 10:  
            processed = cv.bitwise_and(processed, processed, mask=processed_1)

        cv.imshow("out", processed)
        cv.waitKey(0)
        cv.destroyWindow("out")
        
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
        
        elif len(approx) >= 8:
            
            detected_circles = cv.HoughCircles(processed,
                                               cv.HOUGH_GRADIENT, 1, 20, param1=50,
                                               param2=30, minRadius=1, maxRadius=40)

            # Draw circles that are detected.
            if detected_circles is not None:
                print("detected")
                # Convert the circle parameters a, b and r to integers.
                detected_circles = np.uint16(np.around(detected_circles))

                for pt in detected_circles[0, :]:
                    a, b, r = pt[0], pt[1], pt[2]

                    # Draw the circumference of the circle.
                    cv.circle(processed, (a, b), r, (128, 255, 0), 2)

                shapes.append(("circle", cv.contourArea(
                    contour), (cv.boundingRect(contour))))
            
            else :
                if len(approx) == 8:
                    shapes.append(("octagon", cv.contourArea(
                        contour), (cv.boundingRect(contour))))
                    shapes.append("other") 

    return shapes

shape_recognition()


[[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [ 3  1 -1 -1]
  [ 4  2 -1 -1]
  [ 5  3 -1 -1]
  [ 6  4 -1 -1]
  [ 7  5 -1 -1]
  [ 8  6 -1 -1]
  [ 9  7 -1 -1]
  [10  8 -1 -1]
  [11  9 -1 -1]
  [15 10 12 -1]
  [13 -1 -1 11]
  [-1 12 14 11]
  [-1 -1 -1 13]
  [16 11 -1 -1]
  [17 15 -1 -1]
  [18 16 -1 -1]
  [19 17 -1 -1]
  [20 18 -1 -1]
  [21 19 -1 -1]
  [23 20 22 -1]
  [-1 -1 -1 21]
  [25 21 24 -1]
  [-1 -1 -1 23]
  [26 23 -1 -1]
  [-1 25 -1 -1]]]
-1
10
-1
2
-1
2
-1
1
-1
2
-1
2
-1
1
-1
1
-1
10
-1
1
-1
5
-1
8


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'out' in function 'cvDestroyWindow'


In [3]:
import numpy as np

a, b = 1, 1
n = 7
r = 3

y, x = np.ogrid[-a:n-a, -b:n-b]
mask = x*x + y*y <= r*r

array = np.ones((n, n))
array[mask] = 255

print(array)

[[255. 255. 255. 255.   1.   1.   1.]
 [255. 255. 255. 255. 255.   1.   1.]
 [255. 255. 255. 255.   1.   1.   1.]
 [255. 255. 255. 255.   1.   1.   1.]
 [  1. 255.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.]]


In [20]:
import cv2 as cv
import numpy as np

def coords_mouse_disp(event, x, y, flags, param):

    if event == cv.EVENT_LBUTTONDBLCLK:  # left mouse double click
        print("HSV values:", param[y,x])
        s = param.copy()
        cv.circle(s,(x,y), 3,(128),1)
        cv.imshow("eee", s)
        cv.waitKey(0)
        cv.destroyAllWindows()


def apply_segmentation(row):

    img_hist = cv.imread("res\images\\road484.png")
    print(img_hist.shape)
    # TODO - work on histogram equalization
    img_hsv = cv.cvtColor(img_hist, cv.COLOR_BGR2HSV)
    cv.imshow("alo", img_hist)
    cv.setMouseCallback("alo", coords_mouse_disp, img_hsv)
    cv.waitKey(0)
    cv.destroyAllWindows()

    lower_red_m1 = (0, 30, 30)
    upper_red_m1 = (15, 255, 255)

    lower_red_m2 = (150, 30, 30)
    upper_red_m2 = (180, 255, 255)

    red = cv.inRange(img_hsv, lower_red_m1, upper_red_m1) + cv.inRange(img_hsv, lower_red_m2, upper_red_m2)

    cv.imshow("r", red)

    fluorescent_lower = (23, 150,150)
    fluorescent_upper = (40,180,260)
    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 200)
    

    mask1 = cv.inRange(img_hsv, fluorescent_lower, fluorescent_upper)

    blue_mask = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)
    
    # blue
    # apply median filter to remove noise
    blue_mask = cv.medianBlur(blue_mask, 5)

    cv.imshow("alo", blue_mask)
    cv.imshow("m1", mask1)
    cv.waitKey(0)
    cv.destroyAllWindows()
    # out = cv.bitwise_and(img_hist, img_hist, mask=mask)


    out1 = apply_post_processing_blue_test(mask1) 
    out2 = apply_post_processing_blue_test(blue_mask)
    
    if out1 is None: 
        out = out2
    else: 
        out = out1 + out2

    cv.imshow("joi", out)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return img_hist, out


img, processed = apply_segmentation("")
get_contours(img, processed)


(400, 300, 3)
HSV values: [111 102 135]
HSV values: [119  98  83]
HSV values: [119  91  81]
HSV values: [121  99  75]
HSV values: [120  97  79]
HSV values: [120  94  79]
HSV values: [119  99  77]
HSV values: [132 103  57]
HSV values: [131  91  59]
HSV values: [123  96  64]
HSV values: [111  80  99]
HSV values: [127  93  63]
HSV values: [125 103  62]
HSV values: [124  97  63]
HSV values: [124  97  63]
HSV values: [124  96  64]
HSV values: [122 100  66]
HSV values: [124  99  80]
HSV values: [123  99  80]
HSV values: [103 146 236]
HSV values: [102 149 235]
HSV values: [103 150 234]
HSV values: [103 148 233]
HSV values: [105 235  63]
HSV values: [124  94  57]
HSV values: [128  95  59]
HSV values: [128  95  59]
HSV values: [130 101  58]
HSV values: [124  81  69]


NameError: name 'apply_post_processing_blue_test' is not defined

In [ ]:
def apply_post_processing_blue_test(segm_img):

    
    if cv.countNonZero(segm_img) == 0:
        return

    linesP = cv.HoughLinesP(segm_img, 1, np.pi / 180, 50, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(segm_img, (l[0], l[1]), (l[2], l[3]),
                    (255, 0, 255), 3, cv.LINE_AA)

    # blue
    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    rows, cols = out.shape

    cv.imshow("blur", out)

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    cv.imshow("forou", out)
    new = np.ones(out.shape)
    for contour in contours:
        approx = cv.approxPolyDP(
            contour, 0.05*cv.arcLength(contour, True), True)
        print(len(approx))
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if (cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3)):
            new = cv.fillPoly(new, pts=contour, color=(0, 0, 0))
    cv.imshow("forou", out)
    
    mask = np.full(segm_img.shape, 0, "uint8")

    contours, hierarchies = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    # morphological operations
    out = cv.erode(mask, kernel, iterations=3)
    out = cv.dilate(mask, kernel, iterations=4)

    cv.imshow("AA", out)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return out


def get_contours(img, processed):
    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_SIMPLE)
    contours, _ = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(
            contour, 0.01*cv.arcLength(contour, True), True)
        sh = ""
        print("len(approx) =", len(approx))
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "R"
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "O"
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
            sh = "C"

        processed = cv.drawContours(processed, [contour], 0, (128, 0, 255), 2)

        processed = cv.putText(img=processed, text=sh, org=approx[0][0],
                               fontFace=cv.FONT_ITALIC,
                               fontScale=1.0,
                               color=(128, 0, 255),
                               thickness=1)

        cv.imshow("prcessed", processed)
        cv.waitKey(0)
        cv.destroyAllWindows()

    return processed, shapes


## TEST WITH NEW PIPELINE 



In [ ]:
def process_roi(filename, roi):
    #x,y,w,h,cX,cY = roi

    #img = cv.imread('output/histogram/' + filename)
    img = cv.imread('res/images/' + filename)
    #img = img[y:(y+h),x:(x+w)]

    #aux = np.full((h+50, w+50, 3), 0, "uint8")
    #aux[25:25+h, 25:25+w] = img
    #img = aux

    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    lower_red_m1 = (0, 70, 60)
    upper_red_m1 = (10, 255, 255)

    lower_red_m2 = (170, 70, 60)
    upper_red_m2 = (180, 255, 255)

    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 255)

    fluorescent_lower = (23, 150, 150)
    fluorescent_upper = (40, 180, 260)

    mask1 = cv.inRange(img_hsv, lower_red_m1, upper_red_m1)
    mask2 = cv.inRange(img_hsv, lower_red_m2, upper_red_m2)
    mask_red = mask1 + mask2

    cv.imshow(f'Masked red {filename}', mask_red)
    cv.waitKey(0)
    cv.destroyAllWindows()

    mask_blue = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)
    mask_fluorescent = cv.inRange(
        img_hsv, fluorescent_lower, fluorescent_upper)
    mask_blue = mask_blue + mask_fluorescent

    cv.imshow(f'Masked blue {filename}', mask_blue)
    cv.waitKey(0)
    cv.destroyAllWindows()

    ratio_red = cv.countNonZero(mask_red)/(img.size/3)
    ratio_blue = cv.countNonZero(mask_blue)/(img.size/3)

    processed = None
    blue = False
    # morphological operations speciallized
    if ratio_red > ratio_blue:
        processed = mask_red
    else:
        processed = mask_blue
        print(filename)
        blue = True
    # blue
    # apply median filter to remove noise
    processed = cv.medianBlur(processed, 5)
    rows, cols = processed.shape

    cv.imshow("blur", processed)
    # remove small and weird objects

    contours, hierarchy = cv.findContours(
        processed, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            processed = cv.fillPoly(processed, pts=contour, color=(0, 0, 0))

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    processed = cv.morphologyEx(
        processed, cv.MORPH_CLOSE, kernel, iterations=3)

    cv.imshow(f'Processed Before blue {filename}', processed)
    cv.waitKey(0)
    cv.destroyAllWindows()
    if blue:
        #processed = cv.morphologyEx(processed, cv.MORPH_CLOSE, kernel, iterations=3)
        processed = cv.erode(processed, kernel, iterations=2)
        processed = cv.dilate(processed, kernel, iterations=5)

    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_NONE)
    contours1, _ = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]
    contours1 = sorted(contours1, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours1:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(
            contour, 0.01*cv.arcLength(contour, True), True)
        print(len(approx))

        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(
                contour), (cv.boundingRect(contour))))
        # elif len(approx) > 8:
        #     # check if there are hough circles
        #     circles_img = cv.HoughCircles(processed,cv.HOUGH_GRADIENT,1,20,
        #                                   param1=50,
        #                                   param2=30,
        #                                   minRadius=int(w*0.333),
        #                                   maxRadius=0)
        #     if circles_img is not None:
        #         shapes.append(("circle", cv.contourArea(contour), (cv.boundingRect(contour))))

        #elif(len(approx) == 3 ):
         #   tentar limpar o triangulo no meio ou definir como crosswalk

        processed = cv.drawContours(processed, [contour], 0, (128, 0, 255), 2)

        cv.imshow(f'Processed {filename}', processed)
        cv.waitKey(0)
        cv.destroyAllWindows()

    if len(shapes) == 0:
        return 'undefined', ratio_red, ratio_blue

    if shapes[0][0] == 'circle':
        return 'circle', ratio_red, ratio_blue
    elif shapes[0][0] == 'rectangle':
        return 'rectangle', ratio_red, ratio_blue
    elif shapes[0][0] == 'octagon':
        return 'octagon', ratio_red, ratio_blue


process_roi("road324.png", 33)
